## Work
1. 試比較有 BN 在 Batch_size = 2, 16, 32, 128, 256 下的差異
2. 請嘗試將 BN 放在 Activation 之前，並比較訓練結果
3. 請於 BN 放在 Input Layer 後，並比較結果

In [2]:
import os
import keras
import itertools
# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


In [3]:
train, test = keras.datasets.cifar10.load_data()

In [4]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [5]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [6]:
from keras.layers import BatchNormalization, Dropout, Activation
def build_mlp(input_shape, output_units=10, num_neurons=[512, 128, 32], dropout_rate= 0.2, pre_activate= False):
    
    #輸入層
    input_layer= keras.layers.Input(input_shape)
    
    #隱藏層
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x= keras.layers.Dense(units=n_units,                                  
                                  name='hidden_layer'+str(i+1))(input_layer)
            if pre_activate:
                x= BatchNormalization()(x)
                x= Activation("relu")(x)
            else:
                x= Activation("relu")(x)
                x= BatchNormalization()(x)
                
        else:
            x= keras.layers.Dense(units=n_units,
                                  name='hidden_layer'+str(i+1))(x)
            if pre_activate:
                x= BatchNormalization()(x)
                x= Activation("relu")(x)
            else:
                x= Activation("relu")(x)
                x= BatchNormalization()(x)
    
    #輸出層
    output_layer= keras.layers.Dense(units=output_units,
                                     activation='softmax',
                                     name='output_layer')(x)
    
    #建模
    model= keras.models.Model(inputs=[input_layer],outputs=[output_layer])
    return model

In [7]:
Learning_Rate= 1e-3
Epochs= 50
Batch_size= [2, 16, 32, 128, 256] 
pre_activate= [True, False]
Momentum= 0.95

In [ ]:
results = {}
for i, (preact, Bs) in enumerate(itertools.product(pre_activate, Batch_size)):
    print("Numbers of exp: %i, preact: %s, batch_size: %i" % (i, preact, Bs))
    model = build_mlp(input_shape= x_train.shape[1:], pre_activate= preact)
    model.summary()
    opt= keras.optimizers.Adam(lr= Learning_Rate)
    model.compile(loss= 'categorical_crossentropy', metrics=['accuracy'], optimizer= opt)
    model.fit(x_train, y_train,
              epochs= Epochs,
              batch_size= Bs,
              validation_data= (x_test, y_test),
              verbose= 2,
              shuffle= True)
    #collecting data
    name_tag = ('exp-%s-pre-%s-batch_size-%s' % (str(i), str(preact), str(Bs)))
    results[name_tag]= {'train_loss':model.history.history['loss'],
                        'valid_loss':model.history.history['val_loss'],
                        'train_acc':model.history.history['accuracy'],
                        'valid_acc':model.history.history['val_accuracy']}

Numbers of exp: 0, preact: True, batch_size: 2
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
hidden_layer2 (Dense)        (None, 128)               65664     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512     

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
%matplotlib inline

#Rainbow_line
num_colors= len(results.keys())
cm= plt.getcmap('gist_rainbow')
cNorm  = colors.Normalize(vmin=0, vmax=num_colors-1)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)
color_bar = [scalarMap.to_rgba(i) for i in range(num_colors)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train_loss'])),results[cond]['train_loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid_loss'])),results[cond]['valid_loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train_acc'])),results[cond]['train_acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid_acc'])),results[cond]['valid_acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()